In [1]:
import numpy as np

def calculate_max_flow_rate():
    """
    Calculate the maximum flow rate for a non-circular duct with a maximum
    wall shear stress constraint using the hydraulic diameter approach.
    """
    # Define parameters
    width = 20e-3  # Width in meters (20 mm)
    height = 22e-3  # Height in meters (22 mm)
    corner_radius = 3e-3  # Corner radius in meters (3 mm)
    max_shear_stress = 5  # Maximum wall shear stress in Pa
    blood_viscosity = 0.0035  # Blood viscosity at 37°C in Pa·s

    # Calculate cross-sectional area of rounded rectangle
    # A = (W-2R)*(H-2R) + π*R²
    straight_area = (width - 2*corner_radius) * (height - 2*corner_radius)
    corner_area = np.pi * corner_radius**2
    cross_section_area = straight_area + corner_area

    # Calculate wetted perimeter
    # P = 2*(W-2R) + 2*(H-2R) + 2π*R
    straight_perimeter = 2*(width - 2*corner_radius) + 2*(height - 2*corner_radius)
    corner_perimeter = 2 * np.pi * corner_radius
    wetted_perimeter = straight_perimeter + corner_perimeter

    # Calculate hydraulic diameter
    # Dh = 4A/P
    hydraulic_diameter = 4 * cross_section_area / wetted_perimeter

    # Calculate maximum flow rate
    # τ_wall = 8μQ/(A×Dh)
    # Solving for Q: Q = (τ_wall × A × Dh)/(8 × μ)
    max_flow_rate = (max_shear_stress * cross_section_area * hydraulic_diameter) / (8 * blood_viscosity)

    # Calculate average velocity
    # v = Q/A
    average_velocity = max_flow_rate / cross_section_area

    # Convert to different units
    max_flow_rate_ml_s = max_flow_rate * 1e6  # Convert m³/s to mL/s
    max_flow_rate_l_min = max_flow_rate * 60 * 1000  # Convert m³/s to L/min

    # Print results
    print("Geometry:")
    print(f"Width: {width*1000:.1f} mm")
    print(f"Height: {height*1000:.1f} mm")
    print(f"Corner radius: {corner_radius*1000:.1f} mm")
    print(f"Cross-sectional area: {cross_section_area*1e6:.2f} mm²")
    print(f"Wetted perimeter: {wetted_perimeter*1000:.2f} mm")
    print(f"Hydraulic diameter: {hydraulic_diameter*1000:.2f} mm")
    print("\nFlow parameters:")
    print(f"Maximum wall shear stress: {max_shear_stress:.1f} Pa")
    print(f"Blood viscosity: {blood_viscosity:.4f} Pa·s")
    print("\nResults:")
    print(f"Maximum flow rate: {max_flow_rate:.6f} m³/s")
    print(f"Maximum flow rate: {max_flow_rate_ml_s:.2f} mL/s")
    print(f"Maximum flow rate: {max_flow_rate_l_min:.2f} L/min")
    print(f"Average velocity: {average_velocity:.2f} m/s")

    return {
        "area": cross_section_area,
        "hydraulic_diameter": hydraulic_diameter,
        "max_flow_rate_m3_s": max_flow_rate,
        "max_flow_rate_ml_s": max_flow_rate_ml_s,
        "max_flow_rate_l_min": max_flow_rate_l_min,
        "average_velocity": average_velocity
    }

# Execute the calculation
results = calculate_max_flow_rate()

Geometry:
Width: 20.0 mm
Height: 22.0 mm
Corner radius: 3.0 mm
Cross-sectional area: 252.27 mm²
Wetted perimeter: 78.85 mm
Hydraulic diameter: 12.80 mm

Flow parameters:
Maximum wall shear stress: 5.0 Pa
Blood viscosity: 0.0035 Pa·s

Results:
Maximum flow rate: 0.000577 m³/s
Maximum flow rate: 576.53 mL/s
Maximum flow rate: 34.59 L/min
Average velocity: 2.29 m/s


In [2]:
import numpy as np

def calculate_pressure_for_flow_rate(target_flow_rate_l_min, length=0.1):
    """
    Calculate the required wall shear stress and pressure drop for a given flow rate.

    Args:
        target_flow_rate_l_min (float): Target flow rate in L/min
        length (float, optional): Duct length in meters. Defaults to 0.1 m.

    Returns:
        dict: Calculation results including wall shear stress and pressure drop
    """
    # Validate input
    if target_flow_rate_l_min <= 0:
        raise ValueError("Flow rate must be positive")

    # Define parameters
    width = 20e-3  # Width in meters (20 mm)
    height = 22e-3  # Height in meters (22 mm)
    corner_radius = 3e-3  # Corner radius in meters (3 mm)
    blood_viscosity = 0.0035  # Blood viscosity at 37°C in Pa·s
    blood_density = 1060  # Blood density at 37°C in kg/m³

    # Convert target flow rate to m³/s
    # L/min → m³/s conversion: (L/min * 1 m³/1000 L) / (60 s/min)
    target_flow_rate = (target_flow_rate_l_min / 60) * 1e-3

    # Calculate cross-sectional area
    straight_area = (width - 2*corner_radius) * (height - 2*corner_radius)
    corner_area = np.pi * corner_radius**2
    cross_section_area = straight_area + corner_area

    # Calculate wetted perimeter
    straight_perimeter = 2*(width - 2*corner_radius) + 2*(height - 2*corner_radius)
    corner_perimeter = 2 * np.pi * corner_radius
    wetted_perimeter = straight_perimeter + corner_perimeter

    # Calculate hydraulic diameter
    hydraulic_diameter = 4 * cross_section_area / wetted_perimeter

    # Calculate average velocity
    average_velocity = target_flow_rate / cross_section_area

    # Calculate Reynolds number
    reynolds_number = (blood_density * average_velocity * hydraulic_diameter) / blood_viscosity

    # Calculate required wall shear stress for target flow rate
    required_shear_stress = (8 * blood_viscosity * target_flow_rate) / (cross_section_area * hydraulic_diameter)

    # Calculate pressure drop over length L
    pressure_drop = (4 * required_shear_stress * length) / hydraulic_diameter

    # Prepare detailed results
    results = {
        "flow_rate_l_min": target_flow_rate_l_min,
        "flow_rate_m3_s": target_flow_rate,
        "cross_section_area_m2": cross_section_area,
        "hydraulic_diameter_m": hydraulic_diameter,
        "average_velocity_m_s": average_velocity,
        "reynolds_number": reynolds_number,
        "required_shear_stress_pa": required_shear_stress,
        "pressure_drop_pa": pressure_drop
    }

    # Print human-readable results
    print("Geometric Parameters:")
    print(f"Width: {width*1000:.1f} mm")
    print(f"Height: {height*1000:.1f} mm")
    print(f"Hydraulic diameter: {hydraulic_diameter*1000:.2f} mm")
    print(f"Cross-sectional area: {cross_section_area*1e6:.2f} mm²")

    print("\nFlow Parameters:")
    print(f"Flow rate: {target_flow_rate_l_min:.2f} L/min")
    print(f"Average velocity: {average_velocity:.2f} m/s")
    print(f"Reynolds number: {reynolds_number:.2f}")

    print("\nResults:")
    print(f"Required wall shear stress: {required_shear_stress:.2f} Pa")
    #print(f"Pressure drop over {length*1000:.1f} mm: {pressure_drop:.2f} Pa")

    return results

# Example usage
results = calculate_pressure_for_flow_rate(13)

Geometric Parameters:
Width: 20.0 mm
Height: 22.0 mm
Hydraulic diameter: 12.80 mm
Cross-sectional area: 252.27 mm²

Flow Parameters:
Flow rate: 13.00 L/min
Average velocity: 0.86 m/s
Reynolds number: 3328.82

Results:
Required wall shear stress: 1.88 Pa
